In [92]:
import pandas as pd
import openai
import json 
import numpy as np
import ast

In [45]:
with open("OpenAIAPIKey.txt", "r") as f:
    openai.api_key = f.read()

In [46]:
train_data = pd.read_csv("../datasets/dessi-mf/dessi-mf/train.csv")
train_multiclasses = pd.read_csv("../datasets/dessi-mf/dessi-mf/train_labels_multi.csv")
train_classes = pd.read_csv("../datasets/dessi-mf/dessi-mf/train_classes.csv")
train_pii = pd.read_csv("../datasets/dessi-mf/dessi-mf/train_labels_pii.csv")
train_personal = pd.read_csv("../datasets/dessi-mf/dessi-mf/train_labels_personal.csv")
train_dataset = pd.read_csv("../datasets/dessi-mf/dessi-mf/train_dataset.csv")

C:\Users\Luca\AppData\Local\Temp\ipykernel_17844\39221131.py:1: DtypeWarning: Columns (1064,1244,1756,1839,2016,2089,2405,2842,3037,3480,3495,4183,4278,5216,5978,6085,6208,6652,7602,7707,7722,7855,8474,9458,9821,10964,11572,12124,13546,14228,14630,14996,15060,16533,17405,18488,18831,19707,19792,20219,20440,20494,22031,22439,22659,22730,23028,24180,24227,25069,26422,27362,27917,28351,28586,30127,30376,30408,31655,31759,31867,32396,32561,33209,33682,34142,34173,34349,35084,36234,36248,36527,37252,37775,37879,38357,39070,39280,40053,40358,42044,42507,43106) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv("../datasets/dessi-mf/dessi-mf/train.csv")


- Sent a request with the CRSRF-method prompt method of Chinese paper of 2023  

**Capacity and Role**  
"As an accurate classifier of data categories for privacy evaluation,"  
   
**Statement**  
determine whether each provided data class falls into the category of personally identifiable information (PII) or not and evaluate whether the data class could potentially contain personal-related information or not. For each data class (e.g., bank number, email address, or color), assign one of the binary labels: 'pii' or 'non-pi.' and 'personal' or 'non-personal'." 
   
**Reason**   
Identifying PII is critical to ensuring compliance with data protection regulations and safeguarding individuals' privacy and security."  
  
**Format**  
Present your output in a dictionary format with mapping the data classes (keys) to the binary labels (values).  

-->
`As an accurate classifier of data categories for privacy evaluation, determine whether each provided data class falls into the category of personally identifiable information (PII) or not and evaluate whether the data class could potentially contain personal-related information or not. For each data class (e.g., bank number, email address, or color), assign one of the binary labels: 'pii' or 'non-pi.' and 'personal' or 'non-personal'. Identifying PII is critical to ensuring compliance with data protection regulations and safeguarding individuals' privacy and security. Present your output in a dictionary format with mapping the data classes (keys) to the binary labels (values). `

- Provide one or more example like in Chinese paper:    

`You can use the following example as a guideline:`  

`Example Question: first_name: ['Hans', 'Berta', 'Max'], email: contains the email address of a person`  

`Example Answer: {first_name: non-pii, email: pii}, {first_name: personal, email: personal}`

- Sent in the data classes with the SENT method of the paper (seperately instead of all at once)

In [110]:
inital_prompt = """
As an accurate classifier of data categories for privacy evaluation, 
determine whether each provided data class falls into the category of personally identifiable information (PII) or not 
and evaluate whether the data class could potentially contain personal-related information or not. 
PII describes information that can identify a natural person without any additional information.
For each data class (e.g., bank number, email address, or color), assign one of the binary labels: 'pii' or 'non-pi.' and 'personal' or 'non-personal'. 
Identifying PII is critical to ensuring compliance with data protection regulations and safeguarding individuals' privacy and security. 
Present your output in a dictionary format with mapping the data classes (keys) to the binary labels (values). 
Only output this dictionary without any additional information.
"""

example_prompt = """
You can use the following example as a guideline:
Example Question: first_name: ['Hans', 'Berta', 'Max'], email: contains the email address of a person
"""

example_answer = "Example Answer: {first_name: non-pii, email: pii}, {first_name: personal, email: personal}"

conversation = [
    {"role": "system", "content": inital_prompt},
    {"role": "user", "content": example_prompt},
    {"role": "assistant", "content": example_answer}
]

In [111]:
dessi_classes = set([a for a in train_multiclasses["label"] if "," not in a and "Other_data" not in a and "Person" not in a and "Address" not in a])
all_classes = set(dessi_classes)
len(all_classes)

50

In [112]:
combined_df = pd.concat([train_classes, train_pii, train_personal, train_multiclasses, train_dataset], axis=1)
combined_df.columns = ["class", "label_pii", "label_personal", "label_multi", "dataset"]
combined_df

,class,label_pii,label_personal,label_multi,dataset
0,NIN,pii,personal,national_identification_number,dessi
1,programming_language_mixed,non-pii,non-personal,programming_language,mimesis
2,gender_mixed,non-pii,personal,gender,mimesis
3,gender_de,non-pii,personal,gender,mimesis
4,ean_fr_FR,non-pii,non-personal,EAN_code,faker
...,...,...,...,...,...
44210,title_fr,non-pii,personal,academic_degree/title,mimesis
44211,resolution_fr,non-pii,non-personal,resolution,mimesis
44212,iban_de_DE,pii,personal,iban,faker
44213,Phone_number,pii,personal,phone_number,dessi


In [113]:
mapping_prompt = "Now classify the following data classes with the given information:\n"
value_lists = []
for i in all_classes:
    s = i + ": "
    idxs = combined_df.loc[(combined_df["label_multi"] == i)].index
    if combined_df.iloc[idxs[0], 4] != "dessi":
        idx = combined_df.loc[(combined_df["label_multi"] == i) & (combined_df["class"].str.contains("_en"))].index[0]
    else:
        idx = idxs[0]
    val_list = train_data.iloc[:, idx].value_counts().index.tolist()[:10]
    s += str(val_list)
    value_lists.append(s)

for i in range(len(all_classes)):
    mapping_prompt = mapping_prompt + str(value_lists[i]) + "\n"
conversation.append({"role": "user", "content": mapping_prompt})

In [114]:
conversation

[{'role': 'system',
  'content': "\nAs an accurate classifier of data categories for privacy evaluation, \ndetermine whether each provided data class falls into the category of personally identifiable information (PII) or not \nand evaluate whether the data class could potentially contain personal-related information or not. \nPII describes information that can identify a natural person without any additional information.\nFor each data class (e.g., bank number, email address, or color), assign one of the binary labels: 'pii' or 'non-pi.' and 'personal' or 'non-personal'. \nIdentifying PII is critical to ensuring compliance with data protection regulations and safeguarding individuals' privacy and security. \nPresent your output in a dictionary format with mapping the data classes (keys) to the binary labels (values). \nOnly output this dictionary without any additional information.\n"},
 {'role': 'user',
  'content': "\nYou can use the following example as a guideline:\nExample Questi

In [115]:
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=conversation,
    seed=42
)

In [116]:
with open("mapping_results/mapping.json", "w") as f:
    json.dump(response.to_dict(), f)

In [117]:
with open("mapping_results/mapping.json", "rb") as f:
    response_dict = json.load(f)

In [131]:
pii_predictions = ast.literal_eval(response_dict["choices"][0]["message"]["content"][10:1431])
personal_predictions = ast.literal_eval(response_dict["choices"][0]["message"]["content"][1434:-4])

In [132]:
mapping_df = pd.DataFrame(data={
    "class": pii_predictions.keys(),
    "pii manual": [combined_df.loc[combined_df["label_multi"] == i, "label_pii"].values[0] for i in pii_predictions.keys()],
    "pii gpt prediction": pii_predictions.values(),
    "personal manual": [combined_df.loc[combined_df["label_multi"] == i, "label_personal"].values[0] for i in personal_predictions.keys()],
    "personal gpt prediction": personal_predictions.values()
    })
mapping_df.loc[
    (mapping_df["pii gpt prediction"] != mapping_df["pii manual"]) | 
    (mapping_df["personal gpt prediction"] != mapping_df["personal manual"])
]

,class,pii manual,pii gpt prediction,personal manual,personal gpt prediction
6,longitude_and_latitude,pii,non-pii,personal,personal
9,user_agent,non-pii,non-pii,non-personal,personal
14,language,non-pii,non-pii,personal,non-personal
19,SWIFT/BIC code,non-pii,pii,non-personal,personal
35,city,non-pii,non-pii,non-personal,personal


For personal GPT predicts city, language, SWIFT and user_agent different  
SWIFT, city and language are discussable, as it is context dependent (e.g., language can describe the ability of a person or a course in school)  
user_agent seems to be far away from person-related in most contextes  
    
For pii GPT made  different predictions for SWIFT and longitude_and_lattidue  
While longitude_and_lattidue is understandable, as it is not always related to a person, SWIFT can not identify a person